In [1]:
import pandas as pd
import requests as r
from io import BytesIO
import numpy as np
import os
import json

In [15]:
def query(q,url='http://85.193.83.20:8123',format=True,**kwargs):
    if format:
        return pd.read_csv(BytesIO(r.post(url=url,
            data=(q+'\nFORMAT CSVWithNames').encode('utf-8'),
            auth=('crypto_etl','0987654321')).content),**kwargs)
    else:
        return r.post(url=url,
                data=q.encode('utf-8'),
                auth=('crypto_etl','0987654321'))
    

In [21]:
query('''

select

count()

from crypto.du_data

''')

,Code: 60. DB::Exception: Table crypto.du_data doesn't exist. (UNKNOWN_TABLE) (version 22.1.3.7 (official build))


In [4]:
name_dict = {
  "E": 'event_time',
  "s": 'symbol',
  "t": 'trade_id',
  "p": "price",     
  "q": "quantity",       
  "b": 'buyer_order_id',
  "a": 'seller_order_id', 
  "T": 'trade_time', 
  "m": 'is_marker_maker'
}

In [16]:
query('''CREATE TABLE if not exists crypto.trade_data (

                            event_time           UInt64,
                            symbol              String,
                            trade_id           Float64,
                            price              Float64,
                            quantity           Float64,
                            buyer_order_id      UInt64,
                            seller_order_id     UInt64,
                            trade_time          UInt64,
                            is_marker_maker    Float64,
                            type                String
                            ) 
engine = MergeTree
ORDER BY event_time
''').text

''

In [17]:
query('''CREATE TABLE if not exists crypto.du_data (

                            event_time            UInt64,
                            symbol                String,
                            bid_first_q             Float64,
                            ask_first_q             Float64,
                            bid_first_p             Float64,
                            ask_first_p             Float64,
                            diff_p                  Float64,
                            diff_q                  Float64,
                            is_seller_pres_bid    Float64,
                            is_buyer_pres_bid     Float64,
                            is_seller_pres_ask    Float64,
                            is_buyer_pres_ask     Float64

                            ) 
engine = MergeTree
ORDER BY event_time
''').text

''

In [11]:
def upload_du(tmp,db = 'crypto.du_data'):
    tmp['b_first_q']=tmp['b'].explode().dropna().apply(lambda x: x[1] if x[1]!='0.00000000' else np.nan).groupby(level=0).first().astype(float)
    tmp['a_first_q']=tmp['a'].explode().dropna().apply(lambda x: x[1] if x[1]!='0.00000000' else np.nan).groupby(level=0).first().astype(float)

    tmp['b_first_p']=tmp['b'].explode().dropna().apply(lambda x: x[0] if x[1]!='0.00000000' else np.nan).groupby(level=0).first().astype(float)
    tmp['a_first_p']=tmp['a'].explode().dropna().apply(lambda x: x[0] if x[1]!='0.00000000' else np.nan).groupby(level=0).first().astype(float)

    tmp['diff_p']=tmp[['b_first_p','a_first_p']].diff(axis=1).iloc[:,-1]
    tmp['diff_q']=tmp[['b_first_q','a_first_q']].diff(axis=1).iloc[:,-1]

    tmp['is_seller_pres_bid']=tmp['b_first_q'].diff().apply(lambda x: x if x>0 else 0)
    tmp['is_buyer_pres_bid']=tmp[['b_first_q','b_first_p']].diff().apply(lambda x: abs(x['b_first_q']) *-1  if x['b_first_p'] <0 else 0,axis=1)

    tmp['is_seller_pres_ask']=tmp['a_first_q'].diff().apply(lambda x: x if x>0 else 0)
    tmp['is_buyer_pres_ask']=tmp[['a_first_q','a_first_p']].diff().apply(lambda x: abs(x['a_first_q']) *-1  if x['a_first_p'] <0 else 0,axis=1)
    tmp.drop(['e','u','U','b','a'],axis=1,inplace=True)
    tmp=tmp.rename(columns=name_dict)
    
    sample = ','.join(tuple(tmp.apply(lambda x: str(tuple(x)),axis=1).to_list()))
    query(f'INSERT INTO {db} VALUES {sample}')

In [30]:
def upload_trade(tmp,db='crypto.trade_data'):
    tmp=tmp.drop(['e','M'],axis=1)
    tmp['type'] = tmp['p'].astype(float).diff().where(lambda x: x!=0).fillna(method='ffill').mul(100).clip(-1,1).map({-1:'sell',1:'buy'})
    tmp=tmp.rename(columns=name_dict)
    
    sample = ','.join(tuple(tmp.apply(lambda x: str(tuple(x)),axis=1).to_list()))
    query(f'INSERT INTO {db} VALUES {sample}')

In [34]:
with open('data/data.txt') as file:
    n=0
    tmp_list = []
    for i in file:
        n+=1
        try:
                tmp_list.append(json.loads(i))
        except:
                print(f'{n}th line broken')
                pass
        if n % 200000 == 0:
            upload_du(pd.DataFrame(tmp_list).query('e=="depthUpdate"').dropna(axis=1))
            upload_trade(pd.DataFrame(tmp_list).query('e!="depthUpdate"').dropna(axis=1))
            
            print('du_data :',query('''select 
                    max(toDateTime64(event_time/1000,1)) as max,
                    count() as events,
                    uniq(event_time) as uniq
                    from crypto.du_data
                    ''').text)
                    
            print('trade_data :',query('''select 
                    max(toDateTime64(event_time/1000,1)) as max,
                    count() as events,
                    uniq(event_time) as uniq
                    from crypto.trade_data
                    ''').text)

            tmp_list = []

du_data : 2022-01-21 13:12:31.6	81249	80669

trade_data : 1970-01-01 03:00:00.0	0	0

du_data : 2022-01-21 15:31:02.6	163903	162211

trade_data : 2022-01-21 15:31:02.6	117346	74268

du_data : 2022-01-21 16:42:40.8	206856	205657

trade_data : 2022-01-21 15:31:02.6	117346	74268

du_data : 2022-01-21 18:22:12.0	266506	264958

trade_data : 2022-01-21 15:31:02.6	117346	74268

du_data : 2022-01-21 20:21:31.8	338037	336140

trade_data : 2022-01-21 20:21:31.8	245815	155976

du_data : 2022-01-21 22:29:16.2	414508	412418

trade_data : 2022-01-21 22:29:16.2	369344	236348

du_data : 2022-01-22 00:23:44.2	483060	481041

trade_data : 2022-01-22 00:23:44.2	500792	321677

du_data : 2022-01-22 00:59:26.5	504468	502991

trade_data : 2022-01-22 00:59:26.5	679384	417608

du_data : 2022-01-22 01:55:09.6	537862	537684

trade_data : 2022-01-22 01:55:09.7	845990	504197

du_data : 2022-01-22 03:11:23.3	583568	584152

trade_data : 2022-01-22 03:11:23.3	1000284	599647

du_data : 2022-01-22 05:14:12.0	657069	65782

In [11]:
from io import BytesIO

In [15]:
zz=pd.read_csv(BytesIO(query('''select 
                    * from system.tables
                    format CSVWithNames
                    ''').content))

In [18]:
zz['name'].unique()

array(['COLUMNS', 'SCHEMATA', 'TABLES', 'VIEWS', 'binancews_du',
       'binancews_trade', 'du_data', 'trade_data', 'columns', 'schemata',
       'tables', 'views', 'aggregate_function_combinators',
       'asynchronous_inserts', 'asynchronous_metric_log',
       'asynchronous_metrics', 'build_options', 'clusters', 'collations',
       'contributors', 'current_roles', 'data_skipping_indices',
       'data_type_families', 'databases', 'detached_parts',
       'dictionaries', 'disks', 'distributed_ddl_queue',
       'distribution_queue', 'enabled_roles', 'errors', 'events',
       'formats', 'functions', 'grants', 'graphite_retentions',
       'licenses', 'macros', 'merge_tree_settings', 'merges',
       'metric_log', 'metrics', 'models', 'mutations', 'numbers',
       'numbers_mt', 'one', 'part_log', 'part_moves_between_shards',
       'parts', 'parts_columns', 'privileges', 'processes',
       'projection_parts', 'projection_parts_columns', 'query_log',
       'query_thread_log', 'quot